In [14]:
import numpy as np
import pandas as pd

import pymysql
import dotenv
import os
import pickle
import seaborn as sns
import matplotlib

from datetime import timedelta

import matplotlib.pyplot as plt

# Normalization/Standardization
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout, Conv1D, GRU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.saving import save_model

# 경고 무시 코드 추가
import warnings
warnings.filterwarnings('ignore')

def pltconfig_default() :
  sns.reset_defaults()
  %matplotlib inline

pltconfig_default()

matplotlib.rcParams

matplotlib.rcParams['font.family']

current_font_list = matplotlib.rcParams['font.family']

font_path = 'C:\\Windows\\Fonts\\batang.ttc'

kfont = matplotlib.font_manager.FontProperties(fname=font_path).get_name()

matplotlib.rcParams['font.family'] = [kfont] + current_font_list

In [15]:
St_NotEncode_data = pd.read_pickle("StandardScalar_final_data")

print(St_NotEncode_data.shape)

(17422, 9)


In [16]:
# Feature와 Label 분리하기 (상관계수 탑 3 반영 : 일조시간(0.86), 습도(0.51), 지면온도(0.45))
def Feature_Label(datafile) :
    X = datafile.iloc[:,[3,4,7]]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SNE_X, SNE_y = Feature_Label(St_NotEncode_data)
print(SNE_X.shape, SNE_y.shape)
SNE_X_train, SNE_X_test, SNE_y_train, SNE_y_test = train_test_split(SNE_X, SNE_y, test_size=0.2, random_state=10, shuffle=False)
print(SNE_X_train.shape, SNE_X_test.shape, SNE_y_train.shape, SNE_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 3) (17422,)
(13937, 3) (3485, 3) (13937,) (3485,)


In [17]:
# 위 내용을 함수 형태로 만들고 싶은 경우

def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(x).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(y[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [18]:
train_data = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SNE_X_test, SNE_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [19]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 3), (28,)


In [20]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 3]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [21]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 3]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [22]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 3]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [23]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [24]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('new_multi_stne_lstm_weight_top3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('new_multi_stne_lstm_weight_ns_top3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('new_multi_stne_rnn_weight_top3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('new_multi_stne_rnn_weight_ns_top3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('new_multi_stne_gru_weight_top3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('new_multi_stne_gru_weight_ns_top3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [25]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500
    494/Unknown - 20s 10ms/step - loss: 7621.4854 - mse: 183588032.0000 - mae: 7621.8716
Epoch 1: val_loss improved from inf to 7100.58252, saving model to new_multi_stne_lstm_weight_top3.h5
498/498 [==============================] - 24s 18ms/step - loss: 7617.5693 - mse: 183465856.0000 - mae: 7617.9561 - val_loss: 7100.5825 - val_mse: 171084096.0000 - val_mae: 7101.0171 - lr: 0.0010
Epoch 2/500
497/498 [============================>.] - ETA: 0s - loss: 7551.0576 - mse: 180736144.0000 - mae: 7551.4355
Epoch 2: val_loss improved from 7100.58252 to 7024.51709, saving model to new_multi_stne_lstm_weight_top3.h5
498/498 [==============================] - 5s 10ms/step - loss: 7559.2119 - mse: 181011744.0000 - mae: 7559.5894 - val_loss: 7024.5171 - val_mse: 167877328.0000 - val_mae: 7024.9087 - lr: 0.0010
Epoch 3/500
498/498 [==============================] - ETA: 0s - loss: 7453.5903 - mse: 176738368.0000 - mae: 7453.9624
Epoch 3: val_loss improved from 7024.51709 to 6928.75049,

In [26]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500
    492/Unknown - 4s 2ms/step - loss: 7632.2188 - mse: 184055520.0000 - mae: 7632.5879
Epoch 1: val_loss improved from inf to 7100.88037, saving model to new_multi_stne_rnn_weight_top3.h5
498/498 [==============================] - 5s 4ms/step - loss: 7616.6431 - mse: 183407040.0000 - mae: 7617.0132 - val_loss: 7100.8804 - val_mse: 170767568.0000 - val_mae: 7101.3442 - lr: 0.0010
Epoch 2/500
495/498 [============================>.] - ETA: 0s - loss: 7541.8530 - mse: 180356608.0000 - mae: 7542.2310
Epoch 2: val_loss improved from 7100.88037 to 7006.57129, saving model to new_multi_stne_rnn_weight_top3.h5
498/498 [==============================] - 1s 3ms/step - loss: 7546.1099 - mse: 180449952.0000 - mae: 7546.4878 - val_loss: 7006.5713 - val_mse: 167078624.0000 - val_mae: 7006.9531 - lr: 0.0010
Epoch 3/500
481/498 [===========================>..] - ETA: 0s - loss: 7386.9980 - mse: 174446144.0000 - mae: 7387.3682
Epoch 3: val_loss improved from 7006.57129 to 6864.10352, saving

In [27]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500
    494/Unknown - 7s 5ms/step - loss: 7618.6274 - mse: 183805376.0000 - mae: 7618.9927
Epoch 1: val_loss improved from inf to 7107.01416, saving model to new_multi_stne_gru_weight_top3.h5
498/498 [==============================] - 9s 8ms/step - loss: 7620.1621 - mse: 183595216.0000 - mae: 7620.5278 - val_loss: 7107.0142 - val_mse: 171288224.0000 - val_mae: 7107.4150 - lr: 0.0010
Epoch 2/500
493/498 [============================>.] - ETA: 0s - loss: 7572.0737 - mse: 181712336.0000 - mae: 7572.4507
Epoch 2: val_loss improved from 7107.01416 to 7048.82471, saving model to new_multi_stne_gru_weight_top3.h5
498/498 [==============================] - 3s 5ms/step - loss: 7576.4731 - mse: 181783152.0000 - mae: 7576.8501 - val_loss: 7048.8247 - val_mse: 168866176.0000 - val_mae: 7049.1895 - lr: 0.0010
Epoch 3/500
496/498 [============================>.] - ETA: 0s - loss: 7490.2505 - mse: 178292128.0000 - mae: 7490.6206
Epoch 3: val_loss improved from 7048.82471 to 6989.81348, saving

In [28]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 1951.2244873046875
lstm_history Validation MSE: 14496816.0
lstm_history Validation MAE: 1951.5263671875
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 1968.7119140625
lstm_history_ns Validation MSE: 14411305.0
lstm_history_ns Validation MAE: 1969.0325927734375
-------------rnn_history-------------
rnn_history Validation Loss: 1970.021484375
rnn_history Validation MSE: 14955808.0
rnn_history Validation MAE: 1970.3707275390625
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 2122.230712890625
rnn_history_ns Validation MSE: 17189058.0
rnn_history_ns Validation MAE: 2122.58544921875
-------------gru_history-------------
gru_history Validation Loss: 1946.5521240234375
gru_history Validation MSE: 14232911.0
gru_history Validation MAE: 1946.84423828125
-------------gru_history_ns-------------
gru_history_ns Validation Loss: 1980.8624267578125
gru_history_ns Validation MSE: 

In [29]:
lstm_save_path = f"./Models/lstm_model_stne_top3"
rnn_save_path = f"./Models/rnn_model_stne_top3"
gru_save_path = f"./Models/gru_model_stne_top3"

save_model(lstm_model, lstm_save_path, overwrite=True)
save_model(rnn_model, rnn_save_path, overwrite=True)
save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_top3\assets


INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_top3\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_top3\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_top3\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_top3\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_top3\assets


In [30]:
St_NotEncode_Basic_data = pd.read_pickle("Basic_StandardScalar_final_data")
  
print(St_NotEncode_Basic_data.shape)

(52578, 9)


In [31]:
# Feature와 Label 분리하기 (일조시간(0.92), 일사량(0.88), 전운량(0.77))
def Feature_Label(datafile) :
    X = datafile.iloc[:,[4,5,6]]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SB_X, SB_y = Feature_Label(St_NotEncode_Basic_data)
print(SNE_X.shape, SNE_y.shape)
SB_X_train, SB_X_test, SB_y_train, SB_y_test = train_test_split(SB_X, SB_y, test_size=0.2, random_state=10, shuffle=False)
print(SB_X_train.shape, SB_X_test.shape, SB_y_train.shape, SB_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 3) (17422,)
(42062, 3) (10516, 3) (42062,) (10516,)


In [32]:
def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(x).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(y[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [33]:
train_data = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SB_X_test, SB_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [34]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 3), (28,)


In [35]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 3]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [36]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 3]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [37]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 3]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [38]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [39]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('base_multi_stne_lstm_weight_top3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('base_multi_stne_lstm_weight_ns_top3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('base_multi_stne_rnn_weight_top3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('base_multi_stne_rnn_weight_ns_top3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('base_multi_stne_gru_weight_top3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('base_multi_stne_gru_weight_ns_top3.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [40]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500
   1496/Unknown - 12s 5ms/step - loss: 2.0832 - mse: 24.7809 - mae: 2.4458
Epoch 1: val_loss improved from inf to 2.26379, saving model to base_multi_stne_lstm_weight_top3.h5
1503/1503 [==============================] - 14s 6ms/step - loss: 2.0851 - mse: 24.8196 - mae: 2.4478 - val_loss: 2.2638 - val_mse: 23.1942 - val_mae: 2.6491 - lr: 0.0010
Epoch 2/500
1498/1503 [============================>.] - ETA: 0s - loss: 1.3138 - mse: 9.5088 - mae: 1.6603
Epoch 2: val_loss improved from 2.26379 to 2.15116, saving model to base_multi_stne_lstm_weight_top3.h5
1503/1503 [==============================] - 9s 6ms/step - loss: 1.3148 - mse: 9.5251 - mae: 1.6614 - val_loss: 2.1512 - val_mse: 22.1980 - val_mae: 2.5417 - lr: 0.0010
Epoch 3/500
1499/1503 [============================>.] - ETA: 0s - loss: 1.2650 - mse: 9.0194 - mae: 1.6045
Epoch 3: val_loss did not improve from 2.15116
1503/1503 [==============================] - 9s 6ms/step - loss: 1.2654 - mse: 9.0185 - mae: 1.6050 - val_

In [41]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500
   1488/Unknown - 4s 2ms/step - loss: 2.1695 - mse: 26.3971 - mae: 2.5274
Epoch 1: val_loss improved from inf to 2.06314, saving model to base_multi_stne_rnn_weight_top3.h5
1503/1503 [==============================] - 5s 2ms/step - loss: 2.1677 - mse: 26.3288 - mae: 2.5256 - val_loss: 2.0631 - val_mse: 19.8766 - val_mae: 2.4497 - lr: 0.0010
Epoch 2/500
1493/1503 [============================>.] - ETA: 0s - loss: 1.3312 - mse: 9.6758 - mae: 1.6747
Epoch 2: val_loss improved from 2.06314 to 1.95200, saving model to base_multi_stne_rnn_weight_top3.h5
1503/1503 [==============================] - 3s 2ms/step - loss: 1.3367 - mse: 9.7565 - mae: 1.6805 - val_loss: 1.9520 - val_mse: 19.1344 - val_mae: 2.3443 - lr: 0.0010
Epoch 3/500
1489/1503 [============================>.] - ETA: 0s - loss: 1.2907 - mse: 9.2472 - mae: 1.6297
Epoch 3: val_loss did not improve from 1.95200
1503/1503 [==============================] - 3s 2ms/step - loss: 1.2996 - mse: 9.3937 - mae: 1.6389 - val_loss

In [42]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500
   1500/Unknown - 11s 4ms/step - loss: 1.8451 - mse: 18.7602 - mae: 2.1985
Epoch 1: val_loss improved from inf to 2.08535, saving model to base_multi_stne_gru_weight_top3.h5
1503/1503 [==============================] - 13s 5ms/step - loss: 1.8449 - mse: 18.7549 - mae: 2.1983 - val_loss: 2.0854 - val_mse: 20.8107 - val_mae: 2.4592 - lr: 0.0010
Epoch 2/500
1498/1503 [============================>.] - ETA: 0s - loss: 1.3096 - mse: 9.4360 - mae: 1.6511
Epoch 2: val_loss did not improve from 2.08535
1503/1503 [==============================] - 8s 5ms/step - loss: 1.3106 - mse: 9.4453 - mae: 1.6522 - val_loss: 2.2339 - val_mse: 24.2255 - val_mae: 2.6090 - lr: 0.0010
Epoch 3/500
1500/1503 [============================>.] - ETA: 0s - loss: 1.2570 - mse: 8.9869 - mae: 1.5932
Epoch 3: val_loss improved from 2.08535 to 2.06609, saving model to base_multi_stne_gru_weight_top3.h5
1503/1503 [==============================] - 8s 5ms/step - loss: 1.2582 - mse: 9.0014 - mae: 1.5945 - val_lo

In [43]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 1.8475371599197388
lstm_history Validation MSE: 17.681625366210938
lstm_history Validation MAE: 2.2080492973327637
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 1.9494801759719849
lstm_history_ns Validation MSE: 18.565404891967773
lstm_history_ns Validation MAE: 2.3212368488311768
-------------rnn_history-------------
rnn_history Validation Loss: 1.9132683277130127
rnn_history Validation MSE: 18.349205017089844
rnn_history Validation MAE: 2.2695517539978027
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 2.042170763015747
rnn_history_ns Validation MSE: 19.758480072021484
rnn_history_ns Validation MAE: 2.4065914154052734
-------------gru_history-------------
gru_history Validation Loss: 1.788983702659607
gru_history Validation MSE: 16.91541862487793
gru_history Validation MAE: 2.141080379486084
-------------gru_history_ns-------------
gru_history_ns Validation Loss:

In [44]:
lstm_save_path = f"./Models/lstm_model_stne_base_top3"
rnn_save_path = f"./Models/rnn_model_stne_base_top3"
gru_save_path = f"./Models/gru_model_stne_base_top3"

save_model(lstm_model, lstm_save_path, overwrite=True)
save_model(rnn_model, rnn_save_path, overwrite=True)
save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_base_top3\assets


INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_base_top3\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_base_top3\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_base_top3\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_base_top3\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_base_top3\assets
